# Hidden Markov Model Knight's Tour example

A Knight went for a Tour on the chess board.

A drunk reporter wrote down the knight's positions, and we trust each report as having a 50% of being random.
Can we infer where the knight has been?

In [ ]:
%pip install matplotlib numpy

In [ ]:
import itertools
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import sys

sys.path.append(os.path.abspath(".."))

In [ ]:
import knight_tour

path = list(itertools.islice(knight_tour.random_knight_path(), 12))

reported_path = [random.choice([pos, knight_tour.random_position()]) for pos in path]

plt.axis("equal")
plt.plot(*zip(*path), ".-", label="Actual knight tour")
plt.plot(*zip(*reported_path), ".-", label="Reported knight tour", alpha=0.5)
_ = plt.legend()

## Inference using SimProb

In [ ]:
import simprob
import simprob.smoothing as smoothing
import simprob.hidden_markov as hmm


def reported_path_probabilites(pos):
    """
    Position reported by drunk reporter has 50% chance of being random,
    and 50% of being accurate.

    This function computes the resulting probability distribution.
    """
    res = np.ones(knight_tour.BOARD_SHAPE)
    res[pos[::-1]] += res.sum()
    return hmm.Histogram(res)


inferred = np.asarray(
    list(
        smoothing.forward_backward(
            reported_path_probabilites(reported_path[0]),
            [
                simprob.Iteration(
                    transition=hmm.ConvolutionTransition(
                        knight_tour.knight_moves_kernel
                    ),
                    observation=reported_path_probabilites(o),
                )
                for o in reported_path[1:]
            ],
            hmm.Histogram.empty(knight_tour.BOARD_SHAPE),
        )
    )
)

print("Inference of knight's possible positions from observations")
for h, (real_x, real_y), (rep_x, rep_y) in zip(inferred, path, reported_path):
    plt.figure(figsize=(2, 2))
    plt.imshow(h.probs)
    plt.colorbar()
    plt.scatter([real_x], [real_y], color="white")
    plt.scatter([rep_x], [rep_y], marker="x", color="red")
    plt.show()